# コンペティション課題2

## 課題

CNNを用いて、[Tensorflow Flower Dataset](https://www.tensorflow.org/tutorials/load_data/images)のクラス分類を行います。

## 目標値

Accuracy: 80%  
（これはあくまで「目標値」であるため、達成できなかったからといって不合格となったり、著しく成績が損なわれることはありません）

## ルール

- 「修正しないでください」とあるセルを、修正しないでください。
- モデルのアーキテクチャは自由です。講義で扱ったモデル以外でも構いません。
- 訓練データはx_train, t_train、テストデータはx_testで与えられます。
- 予測ラベルは one_hot表現ではなく0~9のクラスラベルによって表してください。
- 以下のセルで定義されているx_train, t_train以外の学習データは使わないでください。

## 提出方法
- 1つのファイルを提出していただきます。
  1. テストデータ`test_iter`に対する予測ラベルを`submission1_pred.csv`として保存・ダウンロードし、Homeworkタブから**Day1 Pred (.csv)**を選択して提出してください。
  2. それとは別に、最終提出に対応するノートブックをわかるようにiLect上に置いておいてください。


- 成績優秀者には、次回講義にて取り組みの発表をお願いいたします。

## 評価方法

- 予測ラベルのt_testに対する精度 (Accuracy) で評価します。

## LeaderBoard

- コンペティション期間中のLeaderBoardは提出されたcsvファイルのうち50%を使って計算されます。コンペティション終了時には提出されたcsvファイルのうち、コンペティション期間中のLeaderBoard計算に使われなかったもう半分のデータがスコア計算に使用されます。
- このため、コンペ中の順位とコンペ終了後にLeaderBoardが更新された後の順位やスコアが食い違うことがあります。

## データの読み込み (仮のコード・そもそもtestデータを作っていない）

- このセルは修正しないでください。
- 誤って修正した場合は、元ファイルをコピーし直してください。

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.autograd as autograd
import torch.nn.functional as F
from torchvision import transforms, datasets, models

In [2]:
class train_dataset(torch.utils.data.Dataset):
    def __init__(self, x_train, t_train, transform):
        self.x_train = x_train
        self.t_train = t_train.astype('int32')
        self.transform = transform

    def __len__(self):
        return self.x_train.shape[0]

    def __getitem__(self, idx):
        data = torch.tensor(self.x_train[idx], dtype=torch.float)
        data = torch.transpose(data,0,2)
        data = torch.transpose(data,1,2)
        label = torch.tensor(self.t_train[idx], dtype=torch.long)
        if self.transform:
            data = self.transform(data)
        return data, label


class test_dataset(torch.utils.data.Dataset):
    def __init__(self, x_test, transform):
        self.x_test = x_test
        self.transform = transform

    def __len__(self):
        return self.x_test.shape[0]

    def __getitem__(self, idx):
        data = torch.tensor(self.x_test[idx], dtype=torch.float)
        data = torch.transpose(data,0,2)
        data = torch.transpose(data,1,2)
        if self.transform:
            data = self.transform(data)
        return data

### 畳み込みニューラルネットワーク(CNN)の実装

In [3]:
print(torch.cuda.get_device_name())

Tesla K80


In [4]:
!nvidia-smi -q -d MEMORY


==============NVSMI LOG==============

Timestamp                                 : Wed Nov 24 23:17:31 2021
Driver Version                            : 450.142.00
CUDA Version                              : 11.0

Attached GPUs                             : 1
GPU 00000000:00:04.0
    FB Memory Usage
        Total                             : 11441 MiB
        Used                              : 3 MiB
        Free                              : 11438 MiB
    BAR1 Memory Usage
        Total                             : 16384 MiB
        Used                              : 2 MiB
        Free                              : 16382 MiB



In [5]:
torch.manual_seed(0) #torchシード固定

In [6]:
import timm
timm.list_models(pretrained=True)

['adv_inception_v3',
 'cait_m36_384',
 'cait_m48_448',
 'cait_s24_224',
 'cait_s24_384',
 'cait_s36_384',
 'cait_xs24_384',
 'cait_xxs24_224',
 'cait_xxs24_384',
 'cait_xxs36_224',
 'cait_xxs36_384',
 'coat_lite_mini',
 'coat_lite_small',
 'coat_lite_tiny',
 'coat_mini',
 'coat_tiny',
 'convit_base',
 'convit_small',
 'convit_tiny',
 'cspdarknet53',
 'cspresnet50',
 'cspresnext50',
 'deit_base_distilled_patch16_224',
 'deit_base_distilled_patch16_384',
 'deit_base_patch16_224',
 'deit_base_patch16_384',
 'deit_small_distilled_patch16_224',
 'deit_small_patch16_224',
 'deit_tiny_distilled_patch16_224',
 'deit_tiny_patch16_224',
 'densenet121',
 'densenet161',
 'densenet169',
 'densenet201',
 'densenetblur121d',
 'dla34',
 'dla46_c',
 'dla46x_c',
 'dla60',
 'dla60_res2net',
 'dla60_res2next',
 'dla60x',
 'dla60x_c',
 'dla102',
 'dla102x',
 'dla102x2',
 'dla169',
 'dm_nfnet_f0',
 'dm_nfnet_f1',
 'dm_nfnet_f2',
 'dm_nfnet_f3',
 'dm_nfnet_f4',
 'dm_nfnet_f5',
 'dm_nfnet_f6',
 'dpn68',
 'dpn

In [7]:
#https://github.com/rwightman/pytorch-image-models/blob/master/results/results-imagenet.csv
model_name = 'vit_small_patch16_224'
model_tuned = timm.create_model(model_name=model_name,pretrained=True)
print(model_tuned)

VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 384, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=384, out_features=1152, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=384, out_features=384, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (drop_path): Identity()
      (norm2): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=384, out_features=1536, bias=True)
        (act): GELU()
        (fc2): Linear(in_features=1536, out_features=384, bias=True)
        (drop): Dropout(p=0.0, inplace=False)
      )
    )
    (1): Block(
      (norm1): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
      (attn

In [8]:
print('変更前 : ', model_tuned.head)

変更前 :  Linear(in_features=384, out_features=1000, bias=True)


In [9]:
model_tuned.head = nn.Linear(in_features=384, out_features=5, bias=True)
print('変更前 : ', model_tuned.head)

変更前 :  Linear(in_features=384, out_features=5, bias=True)


In [10]:
rng = np.random.RandomState(1234)
random_state = 42
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#model_tuned.classifier.apply(init_weights) # 重み・バイアスの初期化を行う
model_tuned.to(device)

VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 384, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=384, out_features=1152, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=384, out_features=384, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (drop_path): Identity()
      (norm2): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=384, out_features=1536, bias=True)
        (act): GELU()
        (fc2): Linear(in_features=1536, out_features=384, bias=True)
        (drop): Dropout(p=0.0, inplace=False)
      )
    )
    (1): Block(
      (norm1): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
      (attn

In [11]:
input_size = timm.get_model_default_value(model_name=model_name,cfg_key="input_size")[1:]
Normalize_mean = timm.get_model_default_value(model_name=model_name,cfg_key="mean")
Normalize_std = timm.get_model_default_value(model_name=model_name,cfg_key="std")

print(input_size)
print(Normalize_mean)
print(Normalize_std)

(224, 224)
(0.5, 0.5, 0.5)
(0.5, 0.5, 0.5)


In [12]:
# %%writefile /root/userspace/submission2_code.py
# 上記のコメントアウトを外し'%%'以下を実行すると、提出コードが出力できます


# 精度向上ポイント: Data Augmentation手法
transform_train =  transforms.Compose([])
transform_test = transforms.Compose([])

# data augmentationとデータ整形

transform_train = transforms.Compose(
    [
         transforms.ToPILImage(),
         transforms.RandomHorizontalFlip(p = 0.5),#左右反転
         #transforms.RandomVerticalFlip(p=0.5),#上下反転
         #transforms.RandomRotation(degrees=30,expand=True),
         transforms.Resize(input_size), # 画像によってサイズが変わってはならない。
         transforms.ToTensor(),
         transforms.Normalize(Normalize_mean, Normalize_std)
    ]
)
transform_test = transforms.Compose(
[
    transforms.ToPILImage(),
    transforms.Resize(input_size),
    transforms.ToTensor(),
    transforms.Normalize(Normalize_mean, Normalize_std)
]
)
x_train = np.load('/root/userspace/public/day2/homework2/data/x_train.npy')
t_train = np.load('/root/userspace/public/day2/homework2/data/t_train.npy')
    
# テストデータ
x_test = np.load('/root/userspace/public/day2/homework2/data/x_test.npy')

trainval_data = train_dataset(x_train, t_train, transform_train)
test_data = test_dataset(x_test,transform_test)
val_ratio = 0.2
val_size = int(len(trainval_data) * val_ratio)
train_size = len(trainval_data) - val_size

train_data, val_data = torch.utils.data.random_split(trainval_data, [train_size, val_size])

# 精度向上ポイント: バッチサイズ
batch_size = 32

dataloader_train = torch.utils.data.DataLoader(
    train_data,
    batch_size=batch_size,
    shuffle=True
)

dataloader_valid = torch.utils.data.DataLoader(
    val_data,
    batch_size=batch_size,
    shuffle=True
)

dataloader_test = torch.utils.data.DataLoader(
    test_data,
    batch_size=batch_size,
    shuffle=False
)

In [13]:
class GAP(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self,x):
        return F.adaptive_avg_pool2d(x,(1,1)).squeeze()

def init_weights(m):  # Heの初期化
    if type(m) == nn.Linear or type(m) == nn.Conv2d:
        torch.nn.init.kaiming_normal_(m.weight)
        m.bias.data.fill_(0.0)

In [14]:
from tqdm import tqdm #プログレスバー

In [15]:
lr = 0.005
optimizer_tuned = optim.SGD(model_tuned.parameters(), lr=lr)
loss_function = nn.CrossEntropyLoss()
n_epochs = 3
for epoch in range(n_epochs):
    losses_train = []
    losses_valid = []

    model_tuned.train()
    n_train = 0
    acc_train = 0
    for x, t in tqdm(dataloader_train):
        n_train += t.size()[0]

        model_tuned.zero_grad()  # 勾配の初期化

        x = x.to(device)  # テンソルをGPUに移動
        t = t.to(device)

        y = model_tuned.forward(x)  # 順伝播

        loss = loss_function(y, t)  # 誤差(クロスエントロピー誤差関数)の計算

        loss.backward()  # 誤差の逆伝播

        optimizer_tuned.step()  # パラメータの更新

        pred = y.argmax(1)  # 最大値を取るラベルを予測ラベルとする

        acc_train += (pred == t).float().sum().item()
        losses_train.append(loss.tolist())

    model_tuned.eval()
    n_val = 0
    acc_val = 0
    for x, t in tqdm(dataloader_valid):
        n_val += t.size()[0]

        x = x.to(device)  # テンソルをGPUに移動
        t = t.to(device)

        y = model_tuned.forward(x)  # 順伝播

        loss = loss_function(y, t)  # 誤差(クロスエントロピー誤差関数)の計算

        pred = y.argmax(1)  # 最大値を取るラベルを予測ラベルとする

        acc_val += (pred == t).float().sum().item()
        losses_valid.append(loss.tolist())

    print('EPOCH: {}, Train [Loss: {:.3f}, Accuracy: {:.3f}], Valid [Loss: {:.3f}, Accuracy: {:.3f}]'.format(
        epoch,
        np.mean(losses_train),
        acc_train/n_train,
        np.mean(losses_valid),
        acc_val/n_val
    ))

  0%|          | 0/87 [00:00<?, ?it/s]

EPOCH: 0, Train [Loss: 0.283, Accuracy: 0.905], Valid [Loss: 0.097, Accuracy: 0.968]


  0%|          | 0/87 [00:00<?, ?it/s]

EPOCH: 1, Train [Loss: 0.058, Accuracy: 0.983], Valid [Loss: 0.082, Accuracy: 0.970]


100%|██████████| 22/22 [00:06<00:00,  3.19it/s]

EPOCH: 2, Train [Loss: 0.026, Accuracy: 0.995], Valid [Loss: 0.084, Accuracy: 0.976]


In [16]:
model_tuned.to(device)
model_tuned.eval()
t_pred = []
for x in dataloader_test:
    x = x.to(device)
    output = model_tuned(x)
    pred = output.argmax(1).tolist()
    t_pred.extend(pred)

submission = pd.Series(t_pred, name='label')
submission.to_csv('/root/userspace/submission2_pred.csv', header=True, index_label='id')

In [17]:
submission

0      3
1      4
2      0
3      3
4      0
      ..
195    1
196    0
197    3
198    2
199    0
Name: label, Length: 200, dtype: int64

In [18]:
#検証は水増ししない
#git_hubでのランキングを参考に(元画像のサイズ・GPUの性能を参考に)ーバッチサイズも調整（スピード重視のため大きく）